In [ ]:
using DataFrames, Dates, Plots
using Turing, FillArrays, StatsPlots, LinearAlgebra, Random, Statistics, CSV
# 1. Load the data into a DataFrame
data = CSV.File("../julia experiments/Data/GB_ENG.csv") |> DataFrame

# 2. Filter out any rows with missing values in the relevant columns
filtered_data = dropmissing(data, [:date, :new_confirmed, :new_deceased, :new_hospitalized_patients, :current_ventilator_patients, :population])

# 3. Convert the 'date' column to Julia Date type
filtered_data.date = Date.(filtered_data.date)

In [ ]:
theme(:bright)
# 5. Plot all the columns on separate subplots in one figure
plot(
    filtered_data.date, 
    [filtered_data.new_confirmed, 
     filtered_data.new_deceased, 
     filtered_data.new_hospitalized_patients, 
     filtered_data.current_ventilator_patients], 
    
    layout = (4, 1), 
    size = (800, 700),
    legend = false,
    title = ["New Confirmed" "New Deceased" "New Hospitalized" "Current Ventilator"],
    ylabel = ["Count" "Count" "Count" "Count"],
    xlabel = "Date",
    lw = 2
)

In [ ]:
data = filtered_data

In [ ]:
t = 1:length(data.new_confirmed)
y_cases = data.new_confirmed

# Standardize the time series
t_min, t_max = extrema(t)
x = (t .- t_min) ./ (t_max - t_min)
y_cases = y_cases .- mean(y_cases)

# Define cyclic features
freqs = 1:10  # For now, considering 10 frequencies, this can be adjusted
num_freqs = length(freqs)
cyclic_features = hcat(sinpi.(2 .* freqs' .* x), cospi.(2 .* freqs' .* x))

# Probabilistic model definition
@model function decomp_model(t, c, op)
    α ~ Normal(0, 10)
    βt ~ Normal(0, 2)
    βc ~ MvNormal(Zeros(size(c, 2)), I)
    σ ~ truncated(Normal(0, 0.1); lower=0)
    cyclic = c * βc
    trend = α .+ βt .* t
    μ = op(trend, cyclic)
    y ~ MvNormal(μ, σ^2 * I)
    return (; trend, cyclic)
end

# Sampling
chain = sample(decomp_model(x, cyclic_features, +) | (; y=y_cases), NUTS(), 2000)